In [1]:
import math
import numpy as np

#MARKER_1 = (1.0, 0.0)
#MARKER_1_OFFSET = 90 # top-right side
#MARKER_2 = (0.0, -1.0)
#MARKER_3_OFFSET = 270 # bottom-left side
#MARKER_3 = (2.0, -1.0)
#MARKER_3_OFFSET = 90 # top-right side

MARKER_1 = (0.0, 0.0)
MARKER_1_OFFSET = 90
MARKER_2 = (1.0, 0.0)
MARKER_2_OFFSET = 90
MARKER_3 = (2.0, 0.0)
MARKER_3_OFFSET = 90

EPS = 1e-16
PRECISION_DIGITS = 5
POSITION_DIGITS = 1

MAX_VALUE = 1e+10

def estimate_line(angle, marker, offset):
    if angle < 0:
        return False, None, None
    m = math.tan(math.radians(offset - angle))
    if m > MAX_VALUE:
        m = float('inf')
    m = round(m, PRECISION_DIGITS)
    n = marker[1] - m * marker[0] if marker[0] > 0 else marker[1]
    return True, m, n

def intersection(line1, line2, marker1, marker2):
    m1, n1 = line1
    m2, n2 = line2

    if m1 == m2:
        print('No intersection')
        return False, None, None

    if m1 == float('inf'):
        x = marker1[0]
        y = m2 * x + n2
    elif m2 == float('inf'):
        x = marker2[0]
        y = m1 * x + n1
    else:
        x = round((n2 - n1) / (m1 - m2 + EPS) if m1 != float('inf') else 1.0, PRECISION_DIGITS)
        y = round(m1 * x + n1 if m1 != float('inf') else m2 * x + n2, PRECISION_DIGITS)

    return True, x, y

# alpha: angle (right hand direction) to spot the 1st marker
# beta:  angle (right hand direction) to spot the 2nd marker
# gamma:  angle (right hand direction) to spot the 3rd marker
# negative angles (-1) indicate the marker was not detected
def estimate_position(alpha, beta, gamma):
    found1, m1, n1 = estimate_line(alpha, MARKER_1, MARKER_1_OFFSET)
    found2, m2, n2 = estimate_line(beta, MARKER_2, MARKER_2_OFFSET)
    found3, m3, n3 = estimate_line(gamma, MARKER_3, MARKER_3_OFFSET)

    print('l1', m1, n1)
    print('l2', m2, n2)
    print('l3', m3, n3)

    x = []
    y = []

    if (found1 and found2):
        intersecting, x1, y1 = intersection((m1, n1), (m2, n2), MARKER_1, MARKER_2)
        if intersecting:
            print('l1 and l2 are intersecting')
            x.append(x1)
            y.append(y1)

    if (found1 and found3):
        intersecting, x2, y2 = intersection((m1, n1), (m3, n3), MARKER_1, MARKER_3)
        if intersecting:
            print('l1 and l3 are intersecting')
            x.append(x2)
            y.append(y2)

    if (found2 and found3):
        intersecting, x3, y3 = intersection((m2, n2), (m3, n3), MARKER_2, MARKER_3)
        if intersecting:
            print('l2 and l3 are intersecting')
            x.append(x3)
            y.append(y3)

    if len(x) == 0 or len(y) ==0:
        raise Exception('No intersections')

    return round(np.mean(x), POSITION_DIGITS), round(np.mean(y), POSITION_DIGITS)

In [2]:
estimate_position(335, -1, 20)

l1 -2.14451 0.0
l2 None None
l3 2.74748 -5.49496
l1 and l3 are intersecting


(1.1, -2.4)

In [174]:
estimate_position(-1, 0, 45)

l1 None None
l2 inf -inf
l3 1.0 -10.0
l2 and l3 are intersecting


(5.0, -5.0)

In [224]:
current_x = 0
current_y = 0

target_x = -0.2
target_y = 0

def get_direction(current_x, current_y, target_x, target_y):
    if target_y >= current_y: # top half
        angle = math.atan((target_x - current_x) / (target_y - current_y + EPS))
        return round(((math.degrees(angle) + 360.0) % 360), 2)
    if target_y < current_y: # bottom half
        angle = math.atan((target_x - current_x) / (target_y - current_y + EPS))
        return round(((180 + math.degrees(angle) + 360.0) % 360), 2)

#0,0 -> 1,0 -> 90
#0,0 -> -1,0 -> 270
#0,0 -> 0,-1 -> 180

print(get_direction(0, 0, 1, 0))
print(get_direction(0, 0, 0, -1))
print(get_direction(0, 0, -1, 0))
print(get_direction(0, 0, 0, 1))
print(get_direction(0, 0, 1, 1))
print(get_direction(0, 0, 1, -1))
print(get_direction(0, 0, -1, -1))
print(get_direction(0, 0, -1, 1))
print(get_direction(0, 0, 2, 1))
print(get_direction(0, 0, 1, -2))
print(get_direction(0, 0, -2, -1))
print(get_direction(0, 0, -1, 2))

90.0
180.0
270.0
0.0
45.0
135.0
225.0
315.0
63.43
153.43
243.43
333.43
